In [1]:
import pandas as pd
#import stockstats as StockDataFrame
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from ipywidgets import *
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rc("figure", figsize=(9, 7))
sns.set_style('whitegrid')

%matplotlib inline
pd.set_option('display.max_columns', 500)
from sklearn.metrics import f1_score
from scipy.optimize import least_squares
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

/home/matthew/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Load stock data

In [2]:
df = pd.read_csv('largecap.csv')

In [3]:
def middle_of_month(name):
    name['date'] = pd.DatetimeIndex(name['date'])
    stockmiddle = name.loc[:,('date','open')]
    stockmiddle['date'] = pd.DatetimeIndex(stockmiddle['date']) + pd.DateOffset(-14)
    stockmiddle['14 day price'] = stockmiddle['open']
    stockmiddle.set_index('date', drop=False, inplace=True)
    stockmiddle.drop('open', 1, inplace=True)
    stock30 = name.loc[:,('date','open')]
    stock30['date'] = pd.DatetimeIndex(stock30['date']) + pd.DateOffset(-28)
    stock30['28 day price'] = stock30['open']
    stock30.set_index('date', drop=False, inplace=True)
    stock30.drop('open', 1, inplace=True)
    stock60 = name.loc[:,('date','open')]
    stock60['date'] = pd.DatetimeIndex(stock60['date']) + pd.DateOffset(-63)
    stock60['63 day price'] = stock60['open']
    stock60.set_index('date', drop=False, inplace=True)
    stock60.drop('open', 1, inplace=True)
    name.set_index('date', drop=False, inplace=True)
    output = pd.merge(name,stockmiddle, left_on='date', right_on='date')
    output = pd.merge(output,stock30, left_on='date', right_on='date')
    output = pd.merge(output,stock60, left_on='date', right_on='date')
    output['2 week gain'] = (output['14 day price'] / output['open']) - 1
    output['4 week gain'] = (output['28 day price'] / output['open']) - 1
    output['7 week gain'] = (output['63 day price'] / output['open']) - 1
    output.set_index('date', drop=False, inplace=True)
    return output

In [4]:
def derivative(data):
    data = data.iloc[1:]
    data['Category: All categories'] = data['Category: All categories'].astype(float)
    data['derivative'] = data['Category: All categories'].diff(periods = 1)
    data.index = [datetime.strptime(date, '%Y-%m-%d').date() for date in data.index]
    data.index = data.index + pd.DateOffset(-2)
    return data
    
def stock(stock,datastock):
    stockdata = df[df['ticker'] == str(stock)]
    datastock = middle_of_month(stockdata)
    return datastock
    
def mergefunction(stock,derivative):
    stock = stock.iloc[1:]
    derivative = pd.merge(stock, derivative,
                          left_index = True,
                          right_index = True,
                          how='inner')
    return stock

In [5]:
df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility,daily_change,dailyrate
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.367758,32.272262,25.817810,28.399591,44739900.0,2.035665e+09,1.968556e+09,-67109850.0,0.222222,-1.50,-0.032967
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,27.714581,27.753307,25.694399,26.062292,10897100.0,4.679215e+08,4.400249e+08,-27896576.0,0.077044,-2.56,-0.059618
2,A,1999-12-29,63.00,79.06,62.94,72.00,7524000.0,0.0,1.0,40.668650,51.035928,40.629918,46.478457,7524000.0,4.740120e+08,5.417280e+08,67716000.0,0.227042,9.00,0.142857
3,A,2000-05-12,84.00,98.00,82.25,90.56,9703800.0,0.0,1.0,54.225854,63.263496,53.096148,58.460635,9703800.0,8.151192e+08,8.787761e+08,63656928.0,0.174757,6.56,0.078095
4,A,2000-05-17,83.25,83.75,76.81,78.00,8906100.0,0.0,1.0,53.739464,54.062223,49.582321,50.350489,8906100.0,7.414328e+08,6.946758e+08,-46757025.0,0.086447,-5.25,-0.063063


In [6]:
tickers = df['ticker'].value_counts()

In [7]:
stocks = pd.DataFrame()
def valuepredictor(df):
    for value in df['ticker']:
        teststock = df[df['ticker'] == value]
        teststock = middle_of_month(teststock)
        value = teststock
        return value
    stocks = pd.merge([value, stocks])

In [8]:
teststock = df[df['ticker'] == 'JNJ']
johnson = middle_of_month(teststock)
johnson.head()

/home/matthew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,marketopen,marketclose,marketgain,volatility,daily_change,dailyrate,14 day price,28 day price,63 day price,2 week gain,4 week gain,7 week gain
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-07-17,JNJ,2001-07-17,53.06,54.98,52.00,54.91,12244400.0,0.0,1.0,35.093997,36.363889,34.392911,36.317591,12244400.0,649687864.0,672340004.0,22652140.0,0.055711,1.85,0.034866,52.92,56.00,54.55,-0.002639,0.055409,0.028081
2001-08-21,JNJ,2001-08-21,54.00,54.94,54.00,54.30,7946000.0,0.0,1.0,35.835321,36.459121,35.835321,36.034406,7946000.0,429084000.0,431467800.0,2383800.0,0.017257,0.30,0.005556,55.75,54.55,58.98,0.032407,0.010185,0.092222
2001-09-05,JNJ,2001-09-05,55.45,56.50,55.25,56.24,11788600.0,0.0,1.0,36.797566,37.494364,36.664842,37.321823,11788600.0,653677870.0,662990864.0,9312994.0,0.022371,0.79,0.014247,54.51,54.45,59.85,-0.016952,-0.018034,0.079351
2001-09-06,JNJ,2001-09-06,56.50,57.16,56.15,56.94,14478600.0,0.0,1.0,37.494364,37.932351,37.262098,37.786355,14478600.0,818040900.0,824411484.0,6370584.0,0.017827,0.44,0.007788,53.24,55.25,59.37,-0.057699,-0.022124,0.050796
2001-09-18,JNJ,2001-09-18,54.55,55.00,53.37,54.50,13398000.0,0.0,1.0,36.200311,36.498938,35.417242,36.167130,13398000.0,730860900.0,730191000.0,-669900.0,0.030082,-0.05,-0.000917,54.55,55.70,59.40,0.000000,0.021082,0.088909


In [9]:
stocks = johnson
for value in df['ticker']:
    teststock = df[df['ticker'] == str(value)]
    teststock = middle_of_month(teststock)
    value = teststock
    stocks = pd.concat([stocks, value])

/home/matthew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


MemoryError: 

In [ ]:
stocks.head()

In [ ]:
stocks['ticker'].value_counts()

# Load fundamental data

In [ ]:
stocks.isnull().sum()

In [ ]:
stocks = stocks.dropna()

In [ ]:
stocks.head()

In [ ]:
X = stocks.drop(['7 week gain','ticker','63 day price','date', 'Category: All categories'], 1)
y = stocks['7 week gain']
reg_scores = cross_val_score(LinearRegression(), X, y, cv=4)
print reg_scores, np.mean(reg_scores)

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, y)
names = X.dtypes.index
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

In [ ]:
# Note the difference in argument order
# optionally, you can chain "fit()" to the model object
model = sm.OLS(y, X)
model = model.fit()
predictions = model.predict(X)

# Plot the model
plt.figure(figsize=(8,6))
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values - $\hat{y}$")
plt.ylabel("Actual Values - $y$")
plt.show()

print "MSE:", mean_squared_error(y, predictions)

In [ ]:
model.summary()